# VacationPy
---

## Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,anadyr,64.7500,177.4833,-15.72,51,92,10.00,RU,1706935968
1,1,yellowknife,62.4560,-114.3525,-12.71,92,100,4.12,CA,1706935969
2,2,port-aux-francais,-49.3500,70.2167,7.29,87,38,5.08,TF,1706935969
3,3,albany,42.6001,-73.9662,-0.51,79,82,0.89,US,1706935715
4,4,emerald,-23.5333,148.1667,35.75,39,5,5.63,AU,1706935969


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.5,
    color = "City"
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
reduced_city_data_df = city_data_df.loc[(city_data_df["Max Temp"] > 21) & (city_data_df["Max Temp"] < 27) ]

reduced_city_data_df = reduced_city_data_df.loc[(reduced_city_data_df["Wind Speed"] < 4.5)]

reduced_city_data_df = reduced_city_data_df.loc[(reduced_city_data_df["Cloudiness"] > 0)]
# Drop any rows with null values
reduced_city_data_df = reduced_city_data_df.dropna()

# Display sample data
reduced_city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
8,8,vila velha,-20.3297,-40.2925,24.97,96,75,1.79,BR,1706935969
9,9,adamstown,-25.0660,-130.1015,24.67,83,42,3.63,PN,1706935970
10,10,port alfred,-33.5906,26.8910,23.53,86,91,3.59,ZA,1706935970
15,15,avarua,-21.2078,-159.7750,25.03,94,100,2.06,CK,1706935970
23,23,ciudad lazaro cardenas,17.9583,-102.2000,23.66,81,5,1.68,MX,1706935971


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = reduced_city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
8,vila velha,BR,-20.3297,-40.2925,96,
9,adamstown,PN,-25.0660,-130.1015,83,
10,port alfred,ZA,-33.5906,26.8910,86,
15,avarua,CK,-21.2078,-159.7750,94,
23,ciudad lazaro cardenas,MX,17.9583,-102.2000,81,
...,...,...,...,...,...,...
545,pizarro,CO,4.9533,-77.3660,95,
548,victorica,AR,-36.2151,-65.4359,31,
557,pimenta bueno,BR,-11.6725,-61.1936,97,
565,natal,BR,-5.7950,-35.2094,94,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey":geoapify_key,
    "categories":"accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search


vila velha - nearest hotel: Hotel Prainha
adamstown - nearest hotel: No hotel found
port alfred - nearest hotel: No hotel found
avarua - nearest hotel: Paradise Inn
ciudad lazaro cardenas - nearest hotel: Hotel Sol del Pacífico
bengkulu - nearest hotel: Dena Hotel
port macquarie - nearest hotel: The Observatory
bredasdorp - nearest hotel: Victoria Hotel
port lincoln - nearest hotel: Boston Hotel
huarmey - nearest hotel: Hostal Santa Rosa
vitorino freire - nearest hotel: No hotel found
tucurui - nearest hotel: No hotel found
hermanus - nearest hotel: Aloe guest house
pariaman - nearest hotel: HOTEL NAN TONGGA
puerto ayora - nearest hotel: Hostal La Mirada De Solitario George
puerto maldonado - nearest hotel: Puerto Palmeras
port saint john's - nearest hotel: Outback Inn
gaoual - nearest hotel: Hotel
port elizabeth - nearest hotel: Waterford Hotel
camiri - nearest hotel: Sol Chaqueño
urucara - nearest hotel: No hotel found
barclayville - nearest hotel: No hotel found
nova sintra - neares

,City,Country,Lat,Lng,Humidity,Hotel Name
8,vila velha,BR,-20.3297,-40.2925,96,Hotel Prainha
9,adamstown,PN,-25.0660,-130.1015,83,No hotel found
10,port alfred,ZA,-33.5906,26.8910,86,No hotel found
15,avarua,CK,-21.2078,-159.7750,94,Paradise Inn
23,ciudad lazaro cardenas,MX,17.9583,-102.2000,81,Hotel Sol del Pacífico
...,...,...,...,...,...,...
545,pizarro,CO,4.9533,-77.3660,95,No hotel found
548,victorica,AR,-36.2151,-65.4359,31,Hotel Mazamet
557,pimenta bueno,BR,-11.6725,-61.1936,97,Cristalina palace hotel
565,natal,BR,-5.7950,-35.2094,94,Natal Palace Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.75,
    color = "City",
    hover_cols = ["Country", "Hotel Name"]
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)